Code and pipeline written by **Matt Mortimer** <br />
Contributions by Matt Spence (see specific scripts)<br />
started 26 Oct 2021<br />
matthew.mortimer@anu.edu.au<br />
Orcid ID: https://orcid.org/0000-0002-8135-9319<br />
Python 3.8<br />

**IMPORTANT: Biopython 1.79 only runs on python 3.6, 3.7 or 3.8**

# Overview - GPAT project
**Background:** <br/>
[Proivde background on the enzyme of interest including the purpose of the SSN]<br/>
Seed sequence length (aa): [insert sequence length]<br/>
[Insert details of protein family and sequence similarity information]<br/>
**Aim:** <br/>
[Insert project aim] <br/> 
**Data sources:**  <br/>
PFAM ([insert pfam id for protein family])

**Dependencies:**<br /> **[Make sure you install these dependencies]** <br/>
1.    Biopython
2.    BioServices
3.    Pandas
4.    Seaborn

**Custom module versions**<br />
[Modules used in this template] <br/>
1. analysis v2.1.0
2. annotations v1.3.2
3. cleaner v1.3.3
4. log v1.2.0
5. progress v1.0.1 
6. run_blast v1.1.2
7. size_filter v1.0.3
8. uniprot v1.0.2
9. utilities v1.0.0

In [ ]:
import pandas as pd

# Custom scripts
from modules.annotations import *
from modules.cleaner import *
from modules.analysis import len_distro
from modules.run_blast import blast
from modules.size_filter import *
from modules.uniprot import uniprot
from modules.utilities import *

In [ ]:
# Set constant variables for this notebook
DATE = datetime.now().strftime('%y%m%d')
PROJECT = '[insert project name here with no spaces]'
PFAM_ID = '[insert PFAM id here]'

# Check working dir, seperately make sure the working dir has 
# two subdirs one called input and another output. Have a sub dir
# in output called BLAST (all capitals)
import pathlib
print(pathlib.Path().resolve())

# Retrieve PFAM family as fasta file

In [ ]:
# The uniprot function takes 3 arguments, the pfam id, and name
# for the full family downloaded fasta file, and then the file name 
# of the filtered dataset containing only sequences from swissprot 
# (reviewed)
output = f"{PROJECT}input/{DATE}_pfam_{PFAM_ID}_all.fasta"
sp_output = f"output/{DATE}_pfam_{PFAM_ID}_swissprot.fasta"

uniprot(PFAM_ID, output, sp_output)

# Sequence assessment and cleaning

In [ ]:
# Take the outputted file from above and pass to len_distro which will provide a 
# seaborn displot, can add bin width and, x and y size as arguments if necessary. 
# This function is used to assess size distribution of sequences which can be 
# filtered if necessary

# Here I've used a binwidth of 10

len_distro(f'{PROJECT}/input/{DATE}_pfam_{PFAM_ID}_all.fasta', 10)

In [ ]:
# This function takes a fasta file, project name, data source as arguments. 
# Stipulate 'greater' or 'less' and the length in resiudes as arguments to 
# filter out others that do not meet that condition. 

# Here I've retained sequences of < than 800 residues in length, discarding 
# the rest.   

size_filter('{PROJECT}/input/{DATE}_pfam_{PFAM_ID}_all.fasta', PROJECT, 'PFAM', 'less', 800)

In [ ]:
# Take the outputted file from above and pass to len_distro which will provide a 
# seaborn displot, can add bin width and, x and y size as arguments if necessary. 
# This function is used to assess size distrobution of sequences which can be 
# filtered if required

# Here I've used a binwidth of 10

len_distro('[insert the file name from the output above]', 10)

In [ ]:
# Based on the distribution above I've taken the output from above and retained 
# sequences < than 360 residues in length, discarding the rest.   

size_filter('[insert the file name from the output above]', PROJECT, 'PFAM', 'less', 360)

In [ ]:
# Based on the distribution above I've taken the output from above and retained 
# sequences > than 240 residues in length, discarding the rest.   

size_filter('[insert the file name from the output above]', PROJECT, 'PFAM', 'greater', 240)

In [ ]:
# Double check the filtering by passing the output from 'size_filter' 
# function back to the len_distro function. 

len_distro('[insert the file name from the output above]', 1)

In [ ]:
# Based on the distribution above I've taken the output from above and retained 
# sequences < than 320 residues in length, discarding the rest.   

size_filter('[insert the file name from the output above]', PROJECT, 'PFAM', 'less', 320)

In [ ]:
# Based on the distribution above I've taken the output from above and retained 
# sequences > than 280 residues in length, discarding the rest.   

size_filter('[insert the file name from the output above]', PROJECT, 'PFAM', 'greater', 280)

In [ ]:
# This function pulls back the UniProt annotations for the sequences 
# filtered by size. Takes 3 arguments, the input file, the data source
# and the project name. Generates a text file in tab seperated csv format

# This function takes a very long time to run, each sequence must be 
# individually passed back to the served and then wait for the reply, 
# sequentially. If the uniprot server is busy it can take longer still.

input_f = '[insert the file name from the output above]'

annotations(input_f, 'PFAM', PROJECT)

# Indexing

In [ ]:
# This indexing function takes the annotation file generated above and 
# the last generated fasta file then adds the sequences to the annotations
#  generating a 'master index' csv file. This can act a reference going 
# forward and a repository of un-edited sequences.

# The function takes 3 arguments, the annotation text file generated from 
# the annotations function, a related fasta file, and the project name. 

# The function works by merging pandas dataframes as 'inner' on the 'Entry'
# name, so if a sequence Entry name is missing from the annotation file
# that sequence is lost.

annotation_file = '[insert name of the annotation .txt file here]'
sequence_file = '[insert name of the file that was input for the annotations function above]'

indexing(annotation_file, sequence_file, PROJECT)

Check data in excel

In [ ]:
# We need to filter out the duplicate sequences, however if one of the
# copies has Status 'reviewed' we need to keep that copy not the 
# unreviewed one. To do this we need to split the dataset into
# reviewed and unreviewed sequences, remove duplicates from within
# those subsets before remove duplicates between the subsets

reviewed('[insert name of mast_index.csv file from above]', 'GPAT')

In [ ]:
# Takes the fasta files generated above and passes them seperately to the
# sequence_cleaner function, with datasource and project name as arguments. 
# Removes duplicate sequences, and sequences with ambigious residues. 
# There maybe duplications between the two files however 

cleaner(f'{PROJECT}/output/{DATE}_reviewed.fasta', 'PFAM_reviewed', PROJECT)

print('') # Line spacer, just makes the output easier to read

cleaner(f'{PROJECT}/output/{DATE}_unreviewed.fasta', 'PFAM_unreviewed', PROJECT)

In [ ]:
reviewed_file = f'{PROJECT}/output/{DATE}_reviewed.fasta'
unreviewed_file = f'{PROJECT}/output/{DATE}_unreviewed.fasta'

reviewed_unreviewed(reviewed_file, unreviewed_file, 'PFAM', PROJECT)

# Generate Network file using blast

In [ ]:
# Run a all v all BLAST search using a blast function 
# BLAST must be installed locally.
# Takes 3 arguments, a fasta file, the project name, and the e-value threshold.

# Just a clear way of laying out the function arguments

in_fasta = 'output/211119_DGAT_PFAM_rev_unrev_deduped.fasta' # Use the last outputted 
                                                        # fasta file from above
# path = '/usr/lib/ncbi-blast-2.12.0+/bin'

# Actual BLAST function, using default E_value_threshold="10e-10", cpus="2" (adjust 
# these arguments as required)

blast(in_fasta, PROJECT, '10e-100', cpus='15')

# Format network file for SSN
The section below uses Pandas to add taxonomic information from the master index created above to the network file. 

In [ ]:
# Set input and output files as variables 
input_network = f'{PROJECT}/output/BLAST/{DATE}_dataset_network_10e-10.csv'
input_index = f'{PROJECT}/output/{DATE}_PFAM_master_index.csv'
output_network = f'{PROJECT}/output/BLAST/{DATE}_SSN_10e-10.csv'

# Read the input files
netwrk = pd.read_csv(input_network)
master_index = pd.read_csv(input_index)

# Make a new dataframe for the input_files with a subset of columns, then rename
# the first column of the input_network dataframe so that it can be easily merged 
# with the index file
nw_mod = netwrk[['Query', 'Target', '% Identity', 'Length', 'E-value', 'Bit-score']]
nw_mod.columns = ['Entry', 'Target', '% Identity', 'Length', 'E-value', 'Bit-score']

index = master_index[['Entry', 'Status', 'Protein names', 'Gene names', 'Organism']]

In [ ]:
# Check the dataframe
nw_mod.head()

In [ ]:
# Check the dataframe
index.head()

In [ ]:
# Merge the dataframes on the Entry column. This will add the 
# taxonomic information to the network file
merged = nw_mod.merge(index, on='Entry', how='left')

In [ ]:
# Check the dataframe, didn't use .head() as need to see the 
# last rows.
merged

In [ ]:
# Write the new network dataframe to file
merged.to_csv(output_network, index=False)